# Buffer System

In [ ]:
class Buffer
  def initialize(max_items, max_flow)
    @ready_pool = 0
    @downstream_buffer = 0

    @max_items = max_items
    @max_flow = max_flow
  end

  def work(items)
    items = [0, items.round].max
    items = discard_excess_items items
    @ready_pool += items

    @downstream_buffer += release_to_buffer

    @downstream_buffer -= work_on_items
    
    @downstream_buffer
  end
  
  private
  
  def discard_excess_items(items)
    [items, @max_items].min
  end
  
  def release_to_buffer
    t = (rand * @ready_pool).round
    @ready_pool -= t
    t
  end
  
  def work_on_items
    r = (rand * @max_flow).round
    [r, @downstream_buffer].min
  end
end

## Hope for the best

Openloop setup described on page 5.

Average units flowing out 

In [ ]:
buffer = Buffer.new 50, 10
data = 1.upto(5000).map do |t|
  u = 5.0 # mean(uniform(0,10))
  y = buffer.work u
  [t, u, 0, u, y]
end.transpose

require 'gnuplot'

Gnuplot::Plot.new do |plot|
  plot.data << Gnuplot::DataSet.new(data.values_at 0, 4) do |ds|
    ds.with = 'lines'
  end
end